In [1]:
# 라이브러리 추가
import requests; from urllib.parse import urlparse
import pandas as pd
import geopandas

In [2]:
# 카카오맵 주소 검색 API: https://developers.kakao.com/docs/restapi/local#주소-검색
address = "강원 원주시 단구로 171"
url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"})
json_obj = result.json()
print(json_obj)

{'documents': [{'x': '127.9477445454879', 'address': {'x': '127.94787953303228', 'address_name': '강원 원주시 명륜동 242-2', 'main_address_no': '242', 'region_3depth_h_name': '명륜1동', 'y': '37.33695203787654', 'region_3depth_name': '명륜동', 'h_code': '4213054100', 'sub_adderss_no': '2', 'region_1depth_name': '강원', 'b_code': '4213010600', 'mountain_yn': 'N', 'sub_address_no': '2', 'main_adderss_no': '242', 'region_2depth_name': '원주시', 'zip_code': '220947'}, 'address_type': 'ROAD_ADDR', 'y': '37.33698555530167', 'road_address': {'main_building_no': '171', 'x': '127.9477445454879', 'address_name': '강원 원주시 단구로 171', 'underground_yn': 'N', 'zone_no': '26441', 'y': '37.33698555530167', 'undergroun_yn': 'N', 'region_3depth_name': '명륜동', 'sub_building_no': '', 'building_name': '국립공원연구원', 'region_1depth_name': '강원', 'road_name': '단구로', 'region_2depth_name': '원주시'}, 'address_name': '강원 원주시 단구로 171'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [3]:
# 검색 결과 정렬
list = []
for document in json_obj['documents']:
    val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    list.append(val)
    
df = pd.DataFrame(list, columns = ['building_name', 'address_name', 'lat', 'lon'])
df

,building_name,address_name,lat,lon
0,국립공원연구원,강원 원주시 단구로 171,37.33698555530167,127.9477445454879


In [4]:
# 주소 목록 xlsx 파일 읽기
df = pd.read_excel('D:/GEODATA/NLPRK_ADR.xlsx', sheet_name='Sheet1')
df

,명칭,주소
0,가야산국립공원사무소,경남 합천군 가야면 가야산로 1200
1,경주국립공원사무소,경북 경주시 천북남로 12
2,계룡산국립공원사무소,충남 공주시 반포면 동학사1로 327-6
3,내장산국립공원사무소,전북 정읍시 내장호반로 328
4,내장산국립공원백암사무소,전남 장성군 북하면 백양로 1116
5,다도해해상국립공원사무소,전남 완도군 완도읍 개포로62번길 17-12
6,다도해해상국립공원서부사무소,전남 목포시 통일대로 102-1
7,덕유산국립공원사무소,전북 무주군 설천면 구천동1로 159
8,무등산국립공원사무소,광주 동구 동산길 29
9,무등산국립공원동부사무소,전남 화순군 화순읍 백운촌길 3-3


In [5]:
# 주소 검색 함수 정의
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 4d25f7ae6c4cab6bc96c746280e403af"})
    json_obj = result.json()
    for document in json_obj['documents']:
        val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    return val

In [6]:
# 주소 검색 함수 적용
address_to_latlon("강원 원주시 단구로 171")

['국립공원연구원', '강원 원주시 단구로 171', '37.33698555530167', '127.9477445454879']

In [7]:
# 주소 검색 함수 반복 적용
list = []
for address in df['주소']:
    list.append(address_to_latlon(address))
    
df1 = pd.DataFrame(list, columns = ['building_name', 'address_name', 'lat', 'lon'])
df1.head()

,building_name,address_name,lat,lon
0,,경남 합천군 가야면 가야산로 1200,35.7679449859344,128.13231846866785
1,,경북 경주시 천북남로 12,35.84377692872769,129.28960201947086
2,,충남 공주시 반포면 동학사1로 327-6,36.356627720879075,127.2325592219709
3,,전북 정읍시 내장호반로 328,35.50408542520653,126.91078956160472
4,내장산국립공원백암사무소,전남 장성군 북하면 백양로 1116,35.42599705536398,126.87927660278751


In [8]:
# 컬럼 값 수정
df1['building_name'] = df['명칭']
df1.head()

,building_name,address_name,lat,lon
0,가야산국립공원사무소,경남 합천군 가야면 가야산로 1200,35.7679449859344,128.13231846866785
1,경주국립공원사무소,경북 경주시 천북남로 12,35.84377692872769,129.28960201947086
2,계룡산국립공원사무소,충남 공주시 반포면 동학사1로 327-6,36.356627720879075,127.2325592219709
3,내장산국립공원사무소,전북 정읍시 내장호반로 328,35.50408542520653,126.91078956160472
4,내장산국립공원백암사무소,전남 장성군 북하면 백양로 1116,35.42599705536398,126.87927660278751


In [9]:
# xlsx 파일로 저장
df1.to_excel('D:/GEODATA/NLPRK_OFF_LATLON.xlsx')

In [10]:
# Shapefile로 저장
geometry=geopandas.points_from_xy(df1.lon.astype(float), df1.lat.astype(float))
gdf = geopandas.GeoDataFrame(df1, geometry=geometry)
gdf.crs= "+init=epsg:4326"
gdf.to_file('D:/GEODATA/NLPRK_OFF_LATLON.shp', driver='ESRI Shapefile', encoding='utf-8')